# Encryption

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from io import StringIO
import cryptography
import ipywidgets as widgets
from ipywidgets import Button, HTML, HBox, VBox, Checkbox, FileUpload, Label, Output, IntSlider, Layout, Image, link, GridspecLayout

In /anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
tools = ["voila", "ipyleaflet", "ipywidgets", "bqplot"]
logos = []
for tool in tools:
    with open(f'/Users/arka.doctor/{tool}.png', 'rb') as f:
        image = f.read()
    img = Image(value=image, format='png',layout=Layout(padding='10px'))
    logos.append(img)
HBox([Label(value='Powered by:')] + logos, layout=Layout(flex_flow='row', align_items='center'))

In [ ]:
import pandas as pd
import sys
import base64
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Protocol.KDF import PBKDF2
from Crypto.Util.Padding import pad, unpad
import os
import time
import getpass

BLOCK_SIZE = 16

def get_private_key_salt(password, salt=None):
    if salt == None:
        saltBin = os.urandom(16)
        salt = saltBin.hex()
    else:                             
        saltBin = bytes.fromhex(salt)    

    kdf = PBKDF2(password, saltBin, 64, 100000)
    key = kdf[:32]

    return [key,salt]


def encrypt(raw, private_key):
    iv = os.urandom(AES.block_size)
    cipher = AES.new(private_key, AES.MODE_CBC, iv)
    return base64.b64encode(iv + cipher.encrypt(pad(raw.encode('utf-8'),BLOCK_SIZE)))


def decrypt(enc, private_key):
    enc = base64.b64decode(enc)
    iv = enc[:16]
    cipher = AES.new(private_key, AES.MODE_CBC, iv)
    #line56
    return unpad(cipher.decrypt(enc[16:]),BLOCK_SIZE)


def encrypt_cols(encryption_dict,password):
    encrypted_dict = {}
    encrypted_dict['salt'] = []
    for dict_key in encryption_dict.keys():
        encrypted_dict[dict_key] = []
        for index in range(0,len(encryption_dict[dict_key])):
            try:
                key_salt_pair = get_private_key_salt(password,encrypted_dict['salt'][index])
            except IndexError:
                key_salt_pair = get_private_key_salt(password)
                encrypted_dict['salt'].append(key_salt_pair[1])
            cipher_text = encrypt(str(encryption_dict[dict_key][index]), key_salt_pair[0])
            encrypted_dict[dict_key].append(cipher_text.decode("utf-8"))

    return encrypted_dict


def decrypt_cols(encrypted_dict,password):
    decrypted_dict = {}
    decrypted_dict['salt'] = encrypted_dict['salt']
    for dict_key in encrypted_dict.keys():
        decrypted_dict[dict_key] = []
        if dict_key != 'salt':
            for index in range(0,len(encrypted_dict[dict_key])):
                key_salt_pair = get_private_key_salt(password,encrypted_dict['salt'][index])
                cipher_text = encrypted_dict[dict_key][index].encode('utf-8')
                #line88
                plain_text = decrypt(cipher_text, key_salt_pair[0]).decode('utf-8')
                decrypted_dict[dict_key].append(plain_text)
    return decrypted_dict


def prep_encryption_cols(encrypt_list):
    encryption_dict = {}
    for col_name in encrypt_list:
        try:
            encryption_dict[col_name]=df[col_name]
        except KeyError:
            print('No column with name \''+col_name+'\' Found. De-identification Failed!')
            sys.exit()
    return encryption_dict

def encryption(df, encrypt_list,password):
    encryption_dict = prep_encryption_cols(encrypt_list)
    encrypted_dict = encrypt_cols(encryption_dict,password)

    for key in encrypted_dict.keys():
        df[key] = encrypted_dict[key]

def decryption(df, decrypt_list, password):
    encrypted_dict = {}
    for col in decrypt_list:
        encrypted_dict[col] = df[col]
    encrypted_dict['salt'] = df['salt']
    decrypted_dict = decrypt_cols(encrypted_dict,password)
    del df['salt']
    for key in decrypted_dict.keys():
        if key != 'salt':
            df[key] = decrypted_dict[key]

password = getpass.getpass(prompt='Type Password Here:')

In [4]:
layout = GridspecLayout(1, 2, height='100px')

def on_upload(change):
    value = change['new']

    for filename in value:
        with open(filename, 'wb') as outfile:
            outfile.write(value[filename]['content'])

upload_data = FileUpload(description='Upload your data:', layout=dict(width='auto', height='auto'))
upload_data.observe(on_upload, 'value')

def show(event):
    df=pd.DataFrame(upload_data.data)
    print(df)
    
    
plot = Button(description='Show', button_style='success', layout=dict(width='auto', height='auto'))
plot.on_click(show)

layout[0, 0] = upload_data
layout[0, 1] = plot

layout

NameError: name 'df' is not defined

In [ ]:
encrypt_list = ['PSU_CODE', 'SAMPLE_CODE','NAME']
encryption(df, encrypt_list, password)
df.to_csv('/Users/arka.doctor/OneDrive/Textile_Study/encrypted_test1.csv')